In [17]:
import pandas as pd
import numpy as np

def load_excel_to_dataframe(file_path, sheet_name=0):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        print(f"Fichier Excel '{file_path}' chargé avec succès dans un DataFrame.")
        #print(f"Les 5 premières lignes du DataFrame :\n{df.head()}")
        return df
    except FileNotFoundError:
        print(f"Erreur : Le fichier '{file_path}' est introuvable. Veuillez vérifier le chemin.")
        return None
    except Exception as e:
        print(f"Une erreur est survenue lors du chargement du fichier Excel : {e}")
        return None

excel_file_reject = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\export devis ECHEC.xls"

df_reject = load_excel_to_dataframe(excel_file_reject)

if df_reject is not None:
    print(f"\nForme du DataFrame : {df_reject.shape}")
    print(f"Colonnes du DataFrame : {df_reject.columns.tolist()}")

excel_file_AO = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\00 SUIVI DES AO.xlsx"

df_AO = load_excel_to_dataframe(excel_file_AO)

if df_AO is not None:
    print(f"\nForme du DataFrame : {df_AO.shape}")
    print(f"Colonnes du DataFrame : {df_AO.columns.tolist()}")


Fichier Excel 'C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\export devis ECHEC.xls' chargé avec succès dans un DataFrame.

Forme du DataFrame : (837, 14)
Colonnes du DataFrame : ['Date', 'Numéro du document', 'CSPS', 'État', 'Ville (livraison)', 'Nom du client', 'Nom (adresse) (livraison)', 'Total HT', 'Durée du chantier en mois', "Montant de l'opération en K€ ", 'Niveau SPS de la mission', "Maître d'oeuvre", 'Montant', 'Retenu']
Fichier Excel 'C:\Users\jch_m\DocumentsPerso\Centrale-IAC\StageATAE\00 SUIVI DES AO.xlsx' chargé avec succès dans un DataFrame.

Forme du DataFrame : (1346, 11)
Colonnes du DataFrame : ['RESULTAT', 'Unnamed: 1', 'DATE', 'VILLE', 'Désignation', 'MOA', 'CSPS', 'Assistante', 'heure', 'Status reponse', 'Unnamed: 10']


In [18]:
from sklearn.model_selection import cross_val_score
from skrub import TableReport
from skrub import tabular_learner

In [19]:
df_AO.info()
df_AO = df_AO.drop(columns=['Unnamed: 1','Unnamed: 10'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1346 entries, 0 to 1345
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   RESULTAT        141 non-null    object
 1   Unnamed: 1      30 non-null     object
 2   DATE            1346 non-null   object
 3   VILLE           1346 non-null   object
 4   Désignation     1340 non-null   object
 5   MOA             1346 non-null   object
 6   CSPS            1341 non-null   object
 7   Assistante      1341 non-null   object
 8   heure           1163 non-null   object
 9   Status reponse  1230 non-null   object
 10  Unnamed: 10     5 non-null      object
dtypes: object(11)
memory usage: 115.8+ KB


In [20]:
# Nettoyage et consolidation des infos

pd.options.mode.copy_on_write = True 

# Consolidation MOA in MOA_full

df_AO['MOA'] = df_AO['MOA'].astype(str)
df_AO['VILLE'] = df_AO['VILLE'].astype(str)
df_AO['MOA_full'] = df_AO['MOA']

for i in range(len(df_AO)):
    current_moa = df_AO.loc[i,('MOA')].strip() # Good practice to strip whitespace
    if current_moa == "MAIRIE": # This covers both "MAIRIE" and "MAIRIE " after stripping
        df_AO.loc[i,('MOA_full')] = "MAIRIE " + df_AO.loc[i]['VILLE']


# Consolidation 'Status reponse' 'Status Rep'

df_AO['Status reponse'] = df_AO['Status reponse'].astype(str)
df_AO['Status Rep'] = df_AO['Status reponse']

for i in range(len(df_AO)):
    current_rep = df_AO.loc[i,('Status reponse')] # Good practice to strip whitespace
    if "ok envoi" in current_rep:
        df_AO.loc[i,('Status Rep')] = "Répondu"
    elif "ne répond pas" in current_rep or "Non répondu" in current_rep or "NON REPONDU" in current_rep:
        df_AO.loc[i,('Status Rep')] = "Non Répondu"
    else :
        df_AO.loc[i,('Status Rep')] = "indéfini"

# Consolidation 'Résultat' 

df_AO['RESULTAT'] = df_AO['RESULTAT'].astype(str)

for i in range(len(df_AO)):
    current_res = df_AO.loc[i,('RESULTAT')] # Good practice to strip whitespace
    if "OUI" in current_res or "RETENU" in current_res or "oui" in current_res:
        df_AO.loc[i,('RESULTAT')] = "Gagné"
    elif "ECHEC" in current_res or "NON" in current_res :
        df_AO.loc[i,('RESULTAT')] = "Perdu"
    else :
        df_AO.loc[i,('RESULTAT')] = "indéfini"


In [21]:
TableReport(df_AO)


Processing column  11 / 11


,,,,,,,,,,,


In [22]:
TableReport(df_reject)

Processing column  14 / 14


,,,,,,,,,,,,,,
